# LSH Banding Technique

In this section, we discuss the more traditional approach to LSH which follows the workflow of ***shingling $\rightarrow$ minhashing $\rightarrow$ banding*** (*the actual LSH step*).

Recall: We can express documents as *k*-shingles (or whichever token we choose) and consequently perform a mminhashing to obtain signatures. We arrange these into columns of a matrix--where rows correspond to each item of a signature. Also, suppose that the signatures now have invariant lengths (i.e., number of rows) due to the prior 'preprocessing' done.

We obtain a structure similar to what is shown below:

```{figure} ./images/band_structure.PNG
:name: band-structure

Segmented signature matrix. Four segments/bands with three rows per segment. Taken from {cite:ps}`rajaraman2011mining`.
```

Notice that unlike those discussed in Minhashing, this signature matrix are now segmented into bands containing three (3) rows per band. Why do this?

Let's break it down. If we simply apply a hash function to full signatures then it will most likely be that we will only get the completely identical signatures--losing pairs that hold some similarity (i.e., candidate pairs) in some segments of their respective signatures. *Note: We end up discarding similar but not identical documents.* This presents another compelling reason for the banded LSH approach.

A natural course to this is to "hash" the items several times using different hash functions banking on the idea that similar items will more likely be hashed to the same bucket--otherwise, dissimilar items. The book terminology for items hashed into the same buckets are *candidate pairs*. Narrowing down the search? Voila! *Candidate pairs* instead of that $n \choose 2$ number of pairs.

Alternatively, for minhashed signatures like the one shown in {numref}`band-structure`, hashing can be applied per band/segment. Hash functions can either be varied per band/segment or the same. In effect, multiple hashing and/or segmented hashing addresses the overfit on getting only identical but not similar items.

Banded signatures are then hashed, forming different hash tables for each band. Candidate pairs are then determined according to those hashed in the same buckets. See {numref}`bandhash-mechanics` for the underlying mechanics.

```{figure} ./images/bandhash_mechanics.PNG
:width: 700px
:name: bandhash-mechanics

Underlying mechanics of the "Band Hashing" method for two banded signature sets $a$ and $b$. Taken from {cite:ps}`band-hashing`.
```